# HMM+viterbi分词

* 定义训练数据目录
* 定义变量
* 定义HMM参数

In [1]:
INPUT_DATA = "RenMinData.txt_utf8"#训练数据集

word_set = set()#词集合
line_num = -1#训练语料行数计数
Count_dic = {}#统计状态出现的词数

A_dic = {}#状态转移概率(状态->状态的条件概率)
B_dic = {}#发射概率(状态->c词的条件概率)
Pi_dic = {}#状态的初始概率
state_list = ['B','M','E','S']#状态值集合

### 初始化HMM参数:

* 状态转移矩阵：A_dic
* 发射矩阵：B_dic
* 初始状态：Pi_dic
* 字频统计器：Count_dic

In [2]:
def init():
    """
    初始化状态转移矩阵、发射矩阵、初始状态矩阵和状态统计词数变量
    """
    for state in state_list:
        A_dic[state] = {}
        for state1 in state_list:
            A_dic[state][state1] = 0.0
    for state in state_list:
        Pi_dic[state] = 0.0
        B_dic[state] = {}
        Count_dic[state] = 0

### 根据分完词的语料，生成标记序列

In [3]:
def getList(input_str):
    """
    根据分完词的语料，生成标记序列
    :param input_str:分完词的语料
    :return content: 标记序列
    """
    outpout_str = []
    if len(input_str) == 1:
        outpout_str.append('S')
    elif len(input_str) == 2:
        outpout_str = ['B','E']
    else:
        M_num = len(input_str) -2
        M_list = ['M'] * M_num
        outpout_str.append('B')
        outpout_str.extend(M_list)
        outpout_str.append('S')
    return outpout_str

In [4]:
def train():
    global word_set
    global line_num
    init()
    with open(INPUT_DATA, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            line = line.strip()#去除空格
            if not line:#去除空行
                continue
            line_num += 1#记录有效行数
            word_list = []#一条样本的词list
            for i in range(len(line)):
                if line[i] == " ":#去除空格
                    continue
                word_list.append(line[i])
            word_set = word_set | set(word_list)#将样本加入词集合

            lineArr = line.split(" ")#分词
            
            #获得每条样本的词标记
            line_state = []
            for item in lineArr:
                line_state.extend(getList(item))
            
            if len(word_list) != len(line_state):#字的个数是否和标记序列的长度一致
                print("[line_num = %d][line = %s]" % (line_num, line))
            else:
                for i in range(len(line_state)):#遍历标记序列
                    if i == 0:
                        #统计：状态的初始概率
                        Pi_dic[line_state[0]] += 1
                        #统计：状态频率
                        Count_dic[line_state[0]] += 1
                    else:
                        #统计：状态转移概率
                        A_dic[line_state[i-1]][line_state[i]] += 1
                        #统计：状态频率
                        Count_dic[line_state[i]] += 1
                        #统计：发射概率
                        if word_list[i] in B_dic[line_state[i]]:
                            B_dic[line_state[i]][word_list[i]] += 1
                        else:
                            B_dic[line_state[i]][word_list[i]] = 1.0
    
    #语料中所有的字数
    print ("len(word_set) = %s " % (len(word_set)))
    
    #词频转换成概率
    
    #状态初始概率
    for key in Pi_dic:
        Pi_dic[key] = Pi_dic[key] * 1.0 / line_num

    #状态转移概率
    for key in A_dic:
        for key1 in A_dic[key]:
            A_dic[key][key1] = A_dic[key][key1] / Count_dic[key]

    #发射概率
    for key in B_dic:
        for word in B_dic[key]:
            B_dic[key][word] = B_dic[key][word] / Count_dic[key]
train()

len(word_set) = 3728 


### 查看初始状态

In [5]:
Pi_dic

{'B': 0.5820149148537713, 'M': 0.0, 'E': 0.0, 'S': 0.41798844132394497}

### 查看状态转移矩阵

In [6]:
A_dic

{'B': {'B': 0.0, 'M': 0.1167175117318146, 'E': 0.8832824882681853, 'S': 0.0},
 'M': {'B': 0.0, 'M': 0.2777743117140081, 'E': 0.0, 'S': 0.7222256882859919},
 'E': {'B': 0.46951648068515556, 'M': 0.0, 'E': 0.0, 'S': 0.5304835193148444},
 'S': {'B': 0.3607655156767958, 'M': 0.0, 'E': 0.0, 'S': 0.47108435638736734}}

### 查看发射矩阵

In [7]:
B_dic["B"]["没"]

0.0017658937640616132

### 维特比算法

In [8]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    path = {}#保存总路径
    for y in states:
        V[0][y] = start_p[y] * emit_p[y][obs[0]]
        path[y] = [y]
    for t in range(1,len(obs)):
        V.append({})
        newpath = {}#保存当前层的全部最优路径
        
        for y in states:#遍历当前层
            prob = 0
            for y0 in states:#遍历前一层
                if V[t-1][y0]>0:
                    if prob < V[t-1][y0] * trans_p[y0][y] * emit_p[y].get(obs[t],0):
                        prob,state = V[t-1][y0] * trans_p[y0][y] * emit_p[y][obs[t]],y0
            V[t][y] =prob
            newpath[y] = path[state] + [y]
        path = newpath
        
    (prob, state) = max([(V[len(obs) - 1][y], y) for y in states])
    return (prob, path[state])

### 分词

In [9]:
def cut(sentence):
    prob, pos_list =  viterbi(sentence,('B','M','E','S'), Pi_dic, A_dic, B_dic)
    new_sentence = ""
    index = 0
    for i in range(len(pos_list)):
        if pos_list[i] == "S":
            new_sentence += sentence[i]
            new_sentence += "  "
        elif pos_list[i] == "B":
            index = i
        elif pos_list[i] == "E":
            new_sentence += sentence[index:i+1]
            new_sentence += "  "
        else:
            pass
 
    return (prob,pos_list,new_sentence)

### 测试

In [10]:
test_str = u"长春市长春节讲话。"
prob,pos_list,cut_words = cut(test_str)
print (test_str)
print (cut_words)
print (pos_list)

test_str = u"他说的确实在理."
prob,pos_list,cut_words = cut(test_str)
print (test_str)
print (cut_words)
print (pos_list)

test_str = u"毛主席万岁。"
prob,pos_list,cut_words = cut(test_str)
print (test_str)
print (cut_words)
print (pos_list)

test_str = u"我有一台电脑。"
prob,pos_list,cut_words = cut(test_str)
print (test_str)
print (cut_words)
print (pos_list)

长春市长春节讲话。
长春  市长  春节  讲话  。  
['B', 'E', 'B', 'E', 'B', 'E', 'B', 'E', 'S']
他说的确实在理.
他  说  的  确实  在  理  .  
['S', 'S', 'S', 'B', 'E', 'S', 'S', 'S']
毛主席万岁。
毛  主席  万  岁  。  
['S', 'B', 'E', 'S', 'S', 'S']
我有一台电脑。
我有  一台  电脑  。  
['B', 'E', 'B', 'E', 'B', 'E', 'S']
